In [2]:
from time_stitches import *
from step1_functions import *
from features import get_jerk
from functions import *
import os
import warnings
warnings.filterwarnings('ignore')
from step1_functions import smooth
from scipy.signal import butter, filtfilt
from scipy.interpolate import CubicSpline

In [3]:
def get_path_length_g(time, pos, new_f = 5):

    dt = 1/new_f + 1/new_f*0.01
    diff_time = np.diff(time, axis=0)
    mask_step_bigger_dt = (diff_time>dt)

    pos_selected = pos[1:][~mask_step_bigger_dt]

    # Calculate the differences in positions between consecutive samples
    differences = np.diff(pos_selected, axis=0)
    # Calculate the Euclidean distance (path length) between consecutive points
    distances = np.linalg.norm(differences, axis=1)
    # Sum up the distances to get the total path length
    total_path_length = np.sum(distances)
    return total_path_length

In [4]:
def compute_v_acc_jerk(data_one_stitch, new_f = 5): #data including time and position
    dt = 1/new_f + 1/new_f*0.01
    diff_t_x_y_z = np.diff(data_one_stitch, axis = 0)
    mask_step_bigger_dt = (diff_t_x_y_z[:,0]>dt)
    t_x_y_z = diff_t_x_y_z[~mask_step_bigger_dt]
    v_x_y_z = [t_x_y_z[:,1]/t_x_y_z[:,0], t_x_y_z[:,2]/t_x_y_z[:,0], t_x_y_z[:,3]/t_x_y_z[:,0]]
    acc_x_y_z = [v_x_y_z[0]/t_x_y_z[:,0], v_x_y_z[1]/t_x_y_z[:,0], v_x_y_z[2]/t_x_y_z[:,0]]
    jerk_x_x_z = [acc_x_y_z[0]/t_x_y_z[:,0], acc_x_y_z[1]/t_x_y_z[:,0], acc_x_y_z[2]/t_x_y_z[:,0]]
    return v_x_y_z, acc_x_y_z, jerk_x_x_z

In [5]:
def get_mean_std_v(v):
    return np.mean(np.linalg.norm(v, axis=0)), np.std(np.linalg.norm(v, axis=0))

In [6]:
def get_list_idle_time(subject, i, list_np_segmented_nh_rec, list_np_segmented_tw_rec, new_f=5):
    needle_holder_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_NH_reconstructed.csv')
    tweezers_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_TW_reconstructed.csv')
    dict_segment_time = full_segments_time[i]
    
    full_time_segment_tw = pd_2_numpy_and_segment(tweezers_rec, dict_segment_time)
    full_time_segment_nh = pd_2_numpy_and_segment(needle_holder_rec, dict_segment_time)

    list_idle_nh = []
    list_idle_tw = []
    for s in range(8):
        effective_t_nh = list_np_segmented_nh_rec[s][:,0]
        effective_t_tw = list_np_segmented_tw_rec[s][:,0]
        tot_time_nh = full_time_segment_nh[s][:,0]
        tot_time_tw = full_time_segment_tw[s][:,0]
        idle_time_nh = tot_time_nh.shape[0] * (1.0 / new_f) - effective_t_nh.shape[0] * (1.0 / new_f)
        idle_time_tw = tot_time_tw.shape[0] * (1.0 / new_f) - effective_t_tw.shape[0] * (1.0 / new_f)
        list_idle_nh.append(idle_time_nh)
        list_idle_tw.append(idle_time_tw)
    return list_idle_nh, list_idle_tw


In [7]:
def get_economy_of_volume(segment_pos, PL):
    x = segment_pos[:,0]
    y = segment_pos[:,1]
    z = segment_pos[:,2]

    EOV = ((np.max(x)-np.min(x))*(np.max(y)-np.min(y))*(np.max(z)-np.min(z)))**(1/3)/PL
    return EOV

In [8]:
def get_curvature(v_,acc_):
    v = np.array(v_)
    acc = np.array(acc_)

    cross = np.cross(v, acc)
    cross_norm = np.linalg.norm(cross, axis =0)

    norm_v = np.linalg.norm(v, axis=0)

    curvature = cross_norm/norm_v**3

    return curvature

In [9]:
def get_mean_std_curvature (v, acc):
    v = np.array(v)
    acc = np.array(acc)
    
    with np.errstate(divide='ignore', invalid='ignore'):
        curvature = np.where(v == 0, np.nan, (v * acc) / (v**3))
    return np.nanmean(curvature), np.nanstd(curvature)

In [10]:
def inverse_quaternion(quaternion):
    q1, q2, q3, q4 = quaternion
    inverse = [q1, -q2, -q3, -q4]/(q1**2 + q2**2 +q3**2 +q4**2) #nécessaire si normalisé???
    return inverse

In [11]:
def get_orientation_change(quaternion_nh, quaternion_tw): #SUPPRIMER UNE FOIS QUE L'AUTRE FONCTIONNE
    #compute unitary quaternion
    norm_quaternion_nh =  quaternion_nh/np.linalg.norm(quaternion_nh, axis=0)
    norm_quaternion_tw =  quaternion_tw/np.linalg.norm(quaternion_tw, axis=0)

    #compute inverse (according to wikipedia formula)
    inverse_nh = []
    for i in range (len(norm_quaternion_nh)):
        inverse_nh.append(inverse_quaternion(norm_quaternion_nh[i,:]))
    inverse_tw = []
    for n in range (len(norm_quaternion_tw)):
        inverse_tw.append(inverse_quaternion(norm_quaternion_tw[n,:]))

    #compute rotation difference
    deltaQ_nh = []
    for j in range (len(norm_quaternion_nh)-1):
        q = np.array(norm_quaternion_nh[j])
        qinv = np.array(inverse_nh[j+1])
        deltaQ_nh.append((q*qinv).tolist())
    
    deltaQ_tw = []
    for j in range (len(norm_quaternion_tw)-1):
        q = np.array(norm_quaternion_tw[j])
        qinv = np.array(inverse_tw[j+1])
        deltaQ_tw.append((q*qinv).tolist())

    #compute orientation change
    deltaTeta_nh = []
    for p in range (len(deltaQ_nh)):
        delta_orientation = 2*np.cos(deltaQ_nh[p][0])**(-1)
        deltaTeta_nh.append(delta_orientation)

    deltaTeta_tw = []
    for p in range (len(deltaQ_tw)):
        delta_orientation = 2*np.cos(deltaQ_tw[p][0])**(-1)
        deltaTeta_tw.append(delta_orientation)
    
    return deltaTeta_nh, deltaTeta_tw
    

In [12]:
def get_orientation_change(quaternion):
    #compute unitary quaternion
    norm_quaternion =  quaternion/np.linalg.norm(quaternion, axis=0)
    
    #compute inverse (according to wikipedia formula)
    inverse = []
    for i in range (len(norm_quaternion)):
        inverse.append(inverse_quaternion(norm_quaternion[i,:]))

    #compute rotation difference
    deltaQ = []
    for j in range (len(norm_quaternion)-1):
        q = np.array(norm_quaternion[j])
        qinv = np.array(inverse[j+1])
        deltaQ.append((q*qinv).tolist())

    #compute orientation change
    deltaTeta = []
    for p in range (len(deltaQ)):
        delta_orientation = 2*np.cos(deltaQ[p][0])**(-1)
        deltaTeta.append(delta_orientation)
    
    return deltaTeta
    

In [13]:
def get_angular_displacement(quaternion):
    deltaTeta = get_orientation_change(quaternion)
    angular_disp = np.sum(np.abs(deltaTeta))
    return angular_disp

In [14]:
def get_rate_of_orientation_change(list_np_segmented, new_f=5):
    time = list_np_segmented[:,0]
    quaternion = list_np_segmented[:, 4:8]
    dt = 1/new_f
    diff_t = np.diff(time, axis = 0)
    mask_step_bigger_dt = (diff_t>dt)
    selected_time = time[1:][~mask_step_bigger_dt]
    selected_quaternion = quaternion[1:][~mask_step_bigger_dt]

    diff_selected_time = np.diff(selected_time, axis = 0)
    deltaTeta = get_orientation_change(selected_quaternion)
    rate_orientation_change = np.mean(np.abs(deltaTeta)/diff_selected_time)

    return rate_orientation_change

In [15]:
def compute_mask_both_tools(subject, i, segment_nh, segment_tw):
    dict_segment_time = full_segments_time[i]
    needle_holder_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_NH_reconstructed.csv')
    tweezers_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_TW_reconstructed.csv')
    ref_nh_time = pd_2_numpy_and_segment(needle_holder_rec, dict_segment_time)[0][0,0]
    ref_tw_time = pd_2_numpy_and_segment(tweezers_rec, dict_segment_time)[0][0,0]

    delta_t = ref_nh_time - ref_tw_time
    time_nh = segment_nh[:,0]
    time_tw = segment_tw[:,0]

    mask_time_nh = np.zeros(len(time_nh))
    for index, t in enumerate(time_nh):
        diff = np.abs(t - np.array(time_tw)) <= (delta_t)
        if (np.sum(diff)>0): mask_time_nh[index]=1

    mask_time_tw = np.zeros(len(time_tw))
    for index, t in enumerate(time_tw):
        diff = np.abs(np.array(time_nh) - t) <= delta_t
        if (np.sum(diff)>0): mask_time_tw[index]=1
    
    return mask_time_nh.astype(int), mask_time_tw.astype(int)

In [16]:
def get_bimanual_dexterity(subject, i, segment_nh, segment_tw):

    v_nh, acc_nh, jerk_nh = compute_v_acc_jerk(segment_nh)
    v_tw, acc_tw, jerk_tw = compute_v_acc_jerk(segment_tw)
    
    mask_nh, mask_tw = compute_mask_both_tools(subject, i, segment_nh, segment_tw)
    
    if ((np.sum(mask_nh)<=5) or np.sum(mask_tw)<=5):
        BD=np.nan
    else: 
        """print('sum', np.sum(mask_nh), np.sum(mask_tw) )
        print(mask_nh)
        print(type(mask_nh))
        print('v', v_nh)
        print(np.array(v_nh).T.shape)"""
        v_nh_selected = np.array(v_nh).T[mask_nh[1:]] #since with the calculation of the velocity we lose the first value, we take our mask starting from the second value
        v_tw_selected = np.array(v_tw).T[mask_tw[1:]]

        v_stitch_nh = np.linalg.norm(v_nh_selected, axis=0)
        v_stitch_tw = np.linalg.norm(v_tw_selected, axis=0)
            
        mean_v_tw = np.nanmean(v_stitch_tw)
        mean_v_nh = np.nanmean(v_stitch_nh)

        BD = np.sum((v_stitch_nh - mean_v_nh)*(v_stitch_tw - mean_v_tw))/np.sqrt(np.sum((v_stitch_nh - mean_v_nh)**2)*np.sum((v_stitch_tw - mean_v_tw)**2))
    
        return BD

    

In [17]:
def get_bimanual_efficiency(subject, i, stitch, segment_nh, segment_tw): #here consider the whole time task (with selected and unselected moments)
    mask_nh, mask_tw = compute_mask_both_tools(subject, i, segment_nh, segment_tw)
    time_nh = segment_nh[:,0][mask_nh.astype(bool)] #since we have a list we need a boolean mask
    diff_nh = np.diff(time_nh)
    sum_time_nh = np.sum(diff_nh)

    dict_segment_time = full_segments_time[i]
    stitch_string = str(stitch+1) #to match the dictionnary
    begin, end = dict_segment_time[stitch_string]
    diff_time = end-begin

    BE= sum_time_nh/diff_time #sum time is the same for nh and tw

    return BE

In [18]:
def get_bimanual_efficiency2(subject, i, stitch, segment_nh, segment_tw, new_f = 5): #consider only time of selected moments
    mask_nh, mask_tw = compute_mask_both_tools(subject, i, segment_nh, segment_tw)
    dt = 1/new_f + 1/new_f*0.01
    time_nh = segment_nh[:,0][mask_nh.astype(bool)] #since we have a list we need a boolean mask
    diff_nh = np.diff(time_nh)
    #we don't want to consider the difference of time between 2 points separated by an unselected period
    mask = diff_nh>dt
    diff_nh_ = diff_nh[~mask]
    sum_time_nh = np.sum(diff_nh_)

    time_whole_segment_nh = segment_nh[:,0]
    diff_whole_segment_nh = np.diff(time_whole_segment_nh)
    mask = diff_whole_segment_nh>dt
    diff_ = diff_whole_segment_nh[~mask]
    sum_time_whole = np.sum(diff_)

    if (sum_time_nh ==0): #it is due to imprecision in the algorithm, we should in theory not have 0 values
        BE=np.nan
    else: 
        BE= sum_time_nh/sum_time_whole #sum time is the same for nh and tw

    return BE

In [19]:
subject = 1
i =0
selected_nh,selected_tw = run_step1_per_subject(subject,i)

In [35]:
dict_segment_time = full_segments_time[i]
needle_holder_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_NH_reconstructed.csv')
tweezers_rec = pd.read_csv(f'Data/Sync_data/S_{subject}_TW_reconstructed.csv')
ref_nh = pd_2_numpy_and_segment(needle_holder_rec, dict_segment_time)
ref_tw = pd_2_numpy_and_segment(tweezers_rec, dict_segment_time)
list_np_segmented_tw_rec = pd_2_numpy_and_segment(selected_tw, dict_segment_time)
list_np_segmented_nh_rec = pd_2_numpy_and_segment(selected_nh, dict_segment_time)

In [21]:
list_np_segmented_nh_rec[0][:,0]
np.ones(len(list_np_segmented_nh_rec[0][:,0]))

array([1., 1., 1., ..., 1., 1., 1.])

In [41]:
for s in range (8):
    BE_old = get_bimanual_efficiency(subject, i, s, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])
    BE_new = get_bimanual_efficiency2(subject, i, s, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])
    print(BE_old, BE_new)

0.8536591643539932 0.46446671273896617
0.8016035841313265 0.3408432270730111
0.7204403370481112 0.26636895402237665
0.6607382073954985 0.15999145573000106
0.8348530872774643 0.16964359206519056
0.7266146993318483 0.17967032967032967
0.7034871085120213 0.19642857142857142
0.420991561181435 0.17235123367575095


In [37]:
for s in range (8):
    BD = get_bimanual_dexterity(subject, i, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])
    print(BD)


0.8458873638219253
0.9209998931515259
0.799584945768617
0.8988483718849074
-0.6290457833592283
-0.20047326624754516
0.7145049337015934
0.577759151908522


In [83]:
def butter_lowpass_filter(data, cutoff=12, fs=120, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

In [20]:
def get_features_simplified(subject_id, i, list_np_segmented_tw_rec, list_np_segmented_nh_rec,new_f=5, save =True):
    # just for the task cluster

    NUMBER_OF_SEGMENTS = 8 

    df_metric = pd.DataFrame(columns=['Tool', 'Stitch','Effective_task_duration', 'Idle_time',  'Path_length', 'Jerk', 'Mean_v', 'Std_v', 'Economy_of_volume',
                                       'Mean_curvature', 'Std_curvature', 'Angular_displacement', 'Rate_of_orientation_change', 'Bimanual_dexterity',
                                       'Bimanual_efficacy'])

    list_idle_nh, list_idle_tw = get_list_idle_time(subject_id, i, list_np_segmented_nh_rec, list_np_segmented_tw_rec)
    for s in range (NUMBER_OF_SEGMENTS):
        t_tw = list_np_segmented_tw_rec[s][:,0]
        t_nh = list_np_segmented_nh_rec[s][:,0]

        idle_t_tw = list_idle_tw[s]
        idle_t_nh = list_idle_nh[s]

        path_length_tw = get_path_length_g(t_tw, list_np_segmented_tw_rec[s][:,1:4])
        path_length_nh = get_path_length_g(t_nh, list_np_segmented_nh_rec[s][:,1:4])

        v_tw, acc_tw, jerk_tw = compute_v_acc_jerk(list_np_segmented_tw_rec[s][:,0:4])
        v_nh, acc_nh, jerk_nh = compute_v_acc_jerk(list_np_segmented_nh_rec[s][:,0:4])

        jerk_approximation_tw = get_jerk(list_np_segmented_tw_rec[s][1:,0], v_tw, acc_tw, jerk_tw) #acc not usefull
        jerk_approximation_nh = get_jerk(list_np_segmented_nh_rec[s][1:,0], v_nh, acc_nh, jerk_nh) #acc not usefull

        mean_v_tw, std_v_tw = get_mean_std_v(v_tw)
        mean_v_nh, std_v_nh = get_mean_std_v(v_nh)

        EOV_nh = get_economy_of_volume(list_np_segmented_nh_rec[s][:,1:4], path_length_nh)
        EOV_tw = get_economy_of_volume(list_np_segmented_tw_rec[s][:,1:4], path_length_tw)

        mean_curvature_nh, std_curvature_nh = get_mean_std_curvature(v_nh, acc_nh)
        mean_curvature_tw, std_curvature_tw = get_mean_std_curvature(v_tw, acc_tw)

        ang_disp_nh= get_angular_displacement(list_np_segmented_nh_rec[s][:, 4:8])
        ang_disp_tw = get_angular_displacement(list_np_segmented_tw_rec[s][:, 4:8])

        rate_orientation_change_nh = get_rate_of_orientation_change(list_np_segmented_nh_rec[s])
        rate_orientation_change_tw = get_rate_of_orientation_change(list_np_segmented_tw_rec[s])
        
        BD = get_bimanual_dexterity(subject_id, i, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])

        BE = get_bimanual_efficiency2(subject, i, s, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])
        
        df_metric.loc[len(df_metric)] = ['TW', s, t_tw.shape[0] * (1.0 / new_f), idle_t_tw,  path_length_tw, jerk_approximation_tw, mean_v_tw, std_v_tw,
                                          EOV_tw, mean_curvature_tw, std_curvature_tw, ang_disp_tw, rate_orientation_change_tw, BD, BE]
        df_metric.loc[len(df_metric)] = ['NH', s, t_nh.shape[0] * (1.0 / new_f), idle_t_nh,  path_length_nh, jerk_approximation_nh, mean_v_nh, std_v_nh,
                                          EOV_nh, mean_curvature_nh, std_curvature_nh, ang_disp_nh, rate_orientation_change_nh, BD, BE]

    if save: 
        print('Save')
        directory = f'Features/S_{subject_id}/'
        if not os.path.exists(directory):
            os.makedirs(directory, exist_ok=True)
        df_metric.to_csv(f'{directory}/df_metrics.csv', index=False)
        return df_metric

    else:
        return df_metric
    

In [21]:
def get_features_simplified_array(subject_id, i, list_np_segmented_tw_rec, list_np_segmented_nh_rec, new_f=5, save =True):
    # just for the task cluster

    NUMBER_OF_SEGMENTS = 8 

    data_metrics = np.zeros((NUMBER_OF_SEGMENTS, 13, 2))
    list_idle_nh, list_idle_tw = get_list_idle_time(subject_id, i, list_np_segmented_nh_rec, list_np_segmented_tw_rec)
    for s in range (NUMBER_OF_SEGMENTS):
        t_tw = list_np_segmented_tw_rec[s][:,0]
        t_nh = list_np_segmented_nh_rec[s][:,0]

        idle_t_tw = list_idle_tw[s]
        idle_t_nh = list_idle_nh[s]

        path_length_tw = get_path_length_g(t_tw, list_np_segmented_tw_rec[s][:,1:4])
        path_length_nh = get_path_length_g(t_nh, list_np_segmented_nh_rec[s][:,1:4])

        v_tw, acc_tw, jerk_tw = compute_v_acc_jerk(list_np_segmented_tw_rec[s][:,0:4])
        v_nh, acc_nh, jerk_nh = compute_v_acc_jerk(list_np_segmented_nh_rec[s][:,0:4])

        jerk_approximation_tw = get_jerk(list_np_segmented_tw_rec[s][1:,0], v_tw, acc_tw, jerk_tw) #acc not usefull
        jerk_approximation_nh = get_jerk(list_np_segmented_nh_rec[s][1:,0], v_nh, acc_nh, jerk_nh) #acc not usefull

        mean_v_tw, std_v_tw = get_mean_std_v(v_tw)
        mean_v_nh, std_v_nh = get_mean_std_v(v_nh)

        EOV_nh = get_economy_of_volume(list_np_segmented_nh_rec[s][:,1:4], path_length_nh)
        EOV_tw = get_economy_of_volume(list_np_segmented_tw_rec[s][:,1:4], path_length_tw)

        mean_curvature_nh, std_curvature_nh = get_mean_std_curvature(v_nh, acc_nh)
        mean_curvature_tw, std_curvature_tw = get_mean_std_curvature(v_tw, acc_tw)

        ang_disp_nh= get_angular_displacement(list_np_segmented_nh_rec[s][:, 4:8])
        ang_disp_tw = get_angular_displacement(list_np_segmented_tw_rec[s][:, 4:8])

        rate_orientation_change_nh = get_rate_of_orientation_change(list_np_segmented_nh_rec[s])
        rate_orientation_change_tw = get_rate_of_orientation_change(list_np_segmented_tw_rec[s])
        
        BD = get_bimanual_dexterity(subject_id, i, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])
        
        BE = get_bimanual_efficiency2(subject, i, s, list_np_segmented_nh_rec[s], list_np_segmented_tw_rec[s])

        data_metrics[s, 0, 0] = t_tw.shape[0] * (1.0 / new_f)
        data_metrics[s, 0, 1] = t_nh.shape[0] * (1.0 / new_f)
        data_metrics[s, 1, 0] = idle_t_tw
        data_metrics[s, 1, 1] = idle_t_nh
        data_metrics[s, 2, 0] = path_length_tw
        data_metrics[s, 2, 1] = path_length_nh
        data_metrics[s, 3, 0] = jerk_approximation_tw
        data_metrics[s, 3, 1] = jerk_approximation_nh
        data_metrics[s, 4, 0] = mean_v_tw
        data_metrics[s, 4, 1] = mean_v_nh
        data_metrics[s, 5, 0] = std_v_tw
        data_metrics[s, 5, 1] = std_v_nh
        data_metrics[s, 6, 0] = EOV_tw
        data_metrics[s, 6, 1] = EOV_nh
        data_metrics[s, 7, 0] = mean_curvature_tw
        data_metrics[s, 7, 1] = mean_curvature_nh
        data_metrics[s, 8, 0] = std_curvature_tw
        data_metrics[s, 8, 1] = std_curvature_nh
        data_metrics[s, 9, 0] = ang_disp_tw
        data_metrics[s, 9, 1] = ang_disp_nh
        data_metrics[s, 10, 0] = rate_orientation_change_tw
        data_metrics[s, 10, 1] = rate_orientation_change_nh
        data_metrics[s, 11, 0] = BD
        data_metrics[s, 11, 1] = BD
        data_metrics[s, 12, 0] = BE
        data_metrics[s, 12, 1] = BE

    if save: 
        print('Save')
        directory = f'Features/S_{subject_id}/'
        if not os.path.exists(directory):
            os.makedirs(directory, exist_ok=True)
        np.save(f"{directory}/ot_metrics.npy", data_metrics)
        return data_metrics

    else:
        return data_metrics
    

In [19]:
subjects=[1,19, 23,7, 24, 26, 10, 13, 16, 17, 20, 27]
i=0
for subject in subjects:
    #select data points corresponding to the main task
    selected_nh,selected_tw = run_step1_per_subject(subject,i)

    #apply filter to coordinates
    selected_nh['X.1'] = butter_lowpass_filter(selected_nh['X.1'], cutoff=12)
    selected_nh['Y.1'] = butter_lowpass_filter(selected_nh['Y.1'], cutoff=12)
    selected_nh['Z.1'] = butter_lowpass_filter(selected_nh['Z.1'], cutoff=12)

    selected_tw['X.1'] = butter_lowpass_filter(selected_tw['X.1'], cutoff=12)
    selected_tw['Y.1'] = butter_lowpass_filter(selected_tw['Y.1'], cutoff=12)
    selected_tw['Z.1'] = butter_lowpass_filter(selected_tw['Z.1'], cutoff=12)

    #cut the time points in stitches
    dict_segment_time = full_segments_time[i]
    list_np_segmented_tw_rec = pd_2_numpy_and_segment(selected_tw, dict_segment_time)
    list_np_segmented_nh_rec = pd_2_numpy_and_segment(selected_nh, dict_segment_time)

    df_metrics = get_features_simplified(subject, i, list_np_segmented_tw_rec, list_np_segmented_nh_rec, save='False')
    print(subject)
    print(df_metrics)
    i=i+1

Save
1
   Tool  Stitch  Effective_task_duration   Idle_time  Path_length       Jerk  \
0    TW       0               238.483333   85.825000     8.336159 -31.280306   
1    NH       0               217.241667  107.058333     4.536104 -31.060091   
2    TW       1               132.166667   87.141667     3.730304 -29.557487   
3    NH       1               133.775000   85.525000     2.431008 -29.540011   
4    TW       2               216.075000  151.833333     4.981386 -30.950042   
5    NH       2               198.858333  169.041667     3.703579 -30.856502   
6    TW       3               149.516667   99.291667     4.199408 -29.779119   
7    NH       3               117.075000  131.725000     2.082637 -28.977462   
8    TW       4               167.433333   62.875000     4.132003 -30.197018   
9    NH       4               151.958333   78.341667     2.616667 -30.028727   
10   TW       5                79.933333   54.775000     2.680974 -27.895020   
11   NH       5                68

In [49]:
subjects=[1,19, 23,7, 24, 26, 10, 13, 16, 17, 20, 27]
i=0
for subject in subjects:
    #select data points corresponding to the main task
    selected_nh,selected_tw = run_step1_per_subject(subject,i)

    #apply filter to coordinates
    selected_nh['X.1'] = butter_lowpass_filter(selected_nh['X.1'], cutoff=12)
    selected_nh['Y.1'] = butter_lowpass_filter(selected_nh['Y.1'], cutoff=12)
    selected_nh['Z.1'] = butter_lowpass_filter(selected_nh['Z.1'], cutoff=12)

    selected_tw['X.1'] = butter_lowpass_filter(selected_tw['X.1'], cutoff=12)
    selected_tw['Y.1'] = butter_lowpass_filter(selected_tw['Y.1'], cutoff=12)
    selected_tw['Z.1'] = butter_lowpass_filter(selected_tw['Z.1'], cutoff=12)

    #cut the time points in stitches
    dict_segment_time = full_segments_time[i]
    list_np_segmented_tw_rec = pd_2_numpy_and_segment(selected_tw, dict_segment_time)
    list_np_segmented_nh_rec = pd_2_numpy_and_segment(selected_nh, dict_segment_time)

    df_metrics = get_features_simplified_array(subject, i, list_np_segmented_tw_rec, list_np_segmented_nh_rec)
    print(subject)
    print(df_metrics)
    i=i+1

Save
1
[[[ 2.38483333e+02  2.17241667e+02]
  [ 8.58250000e+01  1.07058333e+02]
  [ 8.33615941e+00  4.53610438e+00]
  [-3.12803064e+01 -3.10600912e+01]
  [ 3.51440594e-02  2.09103398e-02]
  [ 4.69599992e-02  2.81802085e-02]
  [ 7.39842593e-03  9.06300167e-03]
  [-1.51580993e+05 -6.85349614e+04]
  [ 2.28638544e+07  1.84334893e+07]
  [ 6.01962698e+04  5.44004578e+04]
  [ 1.89183509e+02  1.87639704e+02]
  [-7.06449327e-02 -7.06449327e-02]
  [ 4.64466713e-01  4.64466713e-01]]

 [[ 1.32166667e+02  1.33775000e+02]
  [ 8.71416667e+01  8.55250000e+01]
  [ 3.73030390e+00  2.43100799e+00]
  [-2.95574872e+01 -2.95400106e+01]
  [ 2.86303935e-02  1.83048307e-02]
  [ 4.09558872e-02  1.95659974e-02]
  [ 1.03411396e-02  1.08117230e-02]
  [-1.02514606e+04  4.36846483e+03]
  [ 2.81309694e+06  1.48867316e+06]
  [ 3.35406683e+04  3.34325251e+04]
  [ 1.90290142e+02  1.87325274e+02]
  [-6.92388341e-02 -6.92388341e-02]
  [ 3.40843227e-01  3.40843227e-01]]

 [[ 2.16075000e+02  1.98858333e+02]
  [ 1.51833333e+0

Test curvature

In [22]:
def get_curvature(v_,acc_):

    v = np.stack((v_[0], v_[1], v_[2]), axis=-1)
    v_mm_per_s = v*1000
    acc = np.stack((acc_[0], acc_[1], acc_[2]), axis=-1)
    acc_mm_per_s_squared = acc*1000

    cross = np.cross(v_mm_per_s, acc_mm_per_s_squared)
    cross_norm = np.linalg.norm(cross, axis =1)
    norm_v = np.linalg.norm(v_mm_per_s, axis=1)

    curvature = cross_norm/norm_v**3

    return curvature

In [57]:
subject=1
i=0
#select data points corresponding to the main task
selected_nh,selected_tw = run_step1_per_subject(subject,i)

#apply filter to coordinates
selected_nh['X.1'] = butter_lowpass_filter(selected_nh['X.1'], cutoff=12)
selected_nh['Y.1'] = butter_lowpass_filter(selected_nh['Y.1'], cutoff=12)
selected_nh['Z.1'] = butter_lowpass_filter(selected_nh['Z.1'], cutoff=12)

#cut the time points in stitches
dict_segment_time = full_segments_time[i]
list_np_segmented_nh_rec = pd_2_numpy_and_segment(selected_nh, dict_segment_time)

In [35]:
for s in range (8):
    v_nh, acc_nh, jerk_nh = compute_v_acc_jerk(list_np_segmented_nh_rec[s][:,0:4])

    curvature_mm = get_curvature(v_nh, acc_nh)
    print(np.median(curvature_mm))


3.181452682353221e-16
3.3839484831425915e-16
3.6579989592907124e-16
3.633188747383792e-16
3.577396057883695e-16
2.862015682421637e-16
1.921512259634091e-16
3.4954688961797754e-16


In [36]:
subjects=[1,19, 23,7, 24, 26, 10, 13, 16, 17, 20, 27]
i=0
for subject in subjects:
    #select data points corresponding to the main task
    selected_nh,selected_tw = run_step1_per_subject(subject,i)

    #apply filter to coordinates
    selected_nh['X.1'] = butter_lowpass_filter(selected_nh['X.1'], cutoff=12)
    selected_nh['Y.1'] = butter_lowpass_filter(selected_nh['Y.1'], cutoff=12)
    selected_nh['Z.1'] = butter_lowpass_filter(selected_nh['Z.1'], cutoff=12)

    selected_tw['X.1'] = butter_lowpass_filter(selected_tw['X.1'], cutoff=12)
    selected_tw['Y.1'] = butter_lowpass_filter(selected_tw['Y.1'], cutoff=12)
    selected_tw['Z.1'] = butter_lowpass_filter(selected_tw['Z.1'], cutoff=12)

    #cut the time points in stitches
    dict_segment_time = full_segments_time[i]
    list_np_segmented_tw_rec = pd_2_numpy_and_segment(selected_tw, dict_segment_time)
    list_np_segmented_nh_rec = pd_2_numpy_and_segment(selected_nh, dict_segment_time)

    print(f'Subject {subject}')
    for s in range (8):
        v_nh, acc_nh, jerk_nh = compute_v_acc_jerk(list_np_segmented_nh_rec[s][:,0:4])

        curvature_mm = get_curvature(v_nh, acc_nh)
        print(curvature_mm)

    i = i+1

Subject 1
[4.47828179e-16 2.05073700e-16 0.00000000e+00 ... 3.53040819e-17
 2.22753215e-17 1.53847163e-17]
[2.47448477e-18 2.02779235e-17 5.73263489e-16 ... 1.14139362e-15
 4.72265668e-18 4.90076888e-18]
[8.15142685e-18 4.54884095e-17 0.00000000e+00 ... 5.20773529e-17
 2.33330209e-17 3.34541074e-18]
[3.57899429e-18 1.70769772e-17 7.62116995e-17 ... 1.89276116e-16
 3.15484670e-17 4.07552697e-17]
[3.84263063e-17 2.63491031e-17 1.07681173e-16 ... 3.57345255e-16
 7.04538759e-17 1.79678618e-17]
[3.76881541e-17 1.56598946e-17 0.00000000e+00 ... 0.00000000e+00
 5.46457944e-17 0.00000000e+00]
[3.82361102e-17 4.45724656e-17 2.38797598e-17 ... 0.00000000e+00
 1.74135463e-17 1.52041491e-17]
[1.76799205e-17 1.30036906e-17 1.61623033e-16 ... 0.00000000e+00
 6.39100916e-16 0.00000000e+00]
Subject 19
[1.00399229e-17 1.92892636e-17 2.27852075e-16 ... 2.42072977e-16
 3.59508618e-17 1.31409571e-17]
[4.22792165e-17 0.00000000e+00 8.93459117e-17 ... 3.68221154e-17
 7.39067709e-18 2.08845521e-18]
[6.862947

Test downsampling

In [23]:
def down_sample_cubic_spline(t, x, new_f = 5):
    t_downsampled = np.linspace(t[0], t[-1], int((t[-1] - t[0]) * new_f))  #downsample at 5 Hz
    cs = CubicSpline(t, x)
    downsampled_x= cs(t_downsampled)
    return t_downsampled, downsampled_x

In [24]:
def downsample_recording(tool_signal, new_f=5):
    t = tool_signal['Time (Seconds)'].values
    cols = tool_signal.columns[3:] #we exclude the 2 first unused columns
    new_df = pd.DataFrame()
    for name in cols:
        coordinate = tool_signal[name]
        t_down, coord_downsampled = down_sample_cubic_spline(t,coordinate, new_f = new_f)
        new_df['Time (Seconds)'] = t_down
        new_df[name]=coord_downsampled
    
    return new_df.reset_index(drop=True)

In [25]:
def run_step1_per_subject2 (subject, i, new_f=5 ):
    needle_holder_rec_ = pd.read_csv(f'Data/Sync_data/S_{subject}_NH_reconstructed.csv')
    tweezers_rec_ = pd.read_csv(f'Data/Sync_data/S_{subject}_TW_reconstructed.csv')

    needle_holder_rec = downsample_recording(needle_holder_rec_, new_f = new_f)
    tweezers_rec = downsample_recording(tweezers_rec_, new_f = new_f)

    medians_nh = (needle_holder_rec[['X.1', 'Y.1', 'Z.1']]).median()
    medians_tw = (tweezers_rec[['X.1', 'Y.1', 'Z.1']]).median()

    """X Position"""
    selected_xpos_nh, selected_xpos_tw = remove_extreme_using_x_pos(needle_holder_rec, tweezers_rec, prop_nh=0.5, prop_tw=0.5)
    time_event_xpos_nh, time_event_xpos_tw = compute_event_time_pos(full_segments_time[i], needle_holder_rec, tweezers_rec, selected_xpos_nh,
                                                                selected_xpos_tw)

    pairs_t_xpos_nh = compute_pairs_time(time_event_xpos_nh[0], time_event_xpos_nh[1])
    pairs_t_xpos_tw = compute_pairs_time(time_event_xpos_tw[0], time_event_xpos_tw[1])

    pairs_adjusted_t_xpos_nh = adjust_time_pair(needle_holder_rec, pairs_t_xpos_nh, medians_nh['X.1'], time_allowed = 6)
    pairs_adjusted_t_xpos_tw = adjust_time_pair(tweezers_rec, pairs_t_xpos_tw, medians_tw['X.1'], time_allowed=6)

    mask_extreme_xpos_event_removed_nh = remove_extreme(needle_holder_rec, pairs_adjusted_t_xpos_nh)
    mask_extreme_xpos_event_removed_tw = remove_extreme(tweezers_rec, pairs_adjusted_t_xpos_tw)

    """X Velocity"""
    mask_min_max_nh, mask_min_max_tw = compute_mask_using_v(needle_holder_rec, tweezers_rec, smooth_window=60, prop_nh=0.4, prop_tw=0.4)
    time_event_nh, time_event_tw = compute_event_time(full_segments_time[i], needle_holder_rec, tweezers_rec, mask_min_max_nh[0],  
                                                            mask_min_max_nh[1], mask_min_max_tw[0], mask_min_max_tw[1])
                
    pairs_t_nh = compute_pairs_time(time_event_nh[0], time_event_nh[1])
    pairs_t_tw = compute_pairs_time(time_event_tw[0], time_event_tw[1])

    pairs_adjusted_t_nh = adjust_time_pair(needle_holder_rec, pairs_t_nh, medians_nh['X.1'], time_allowed = 6)
    pairs_adjusted_t_tw = adjust_time_pair(tweezers_rec, pairs_t_tw, medians_tw['X.1'], time_allowed = 6)

    mask_extreme_v_event_removed_nh = remove_extreme(needle_holder_rec, pairs_adjusted_t_nh)
    mask_extreme_v_event_removed_tw = remove_extreme(tweezers_rec, pairs_adjusted_t_tw)
    mask_extreme_v_glass_nh = remove_extreme(needle_holder_rec, pairs_adjusted_t_tw) #we don't want to keep event when the other tool is in the glass
    mask_extreme_v_glass_tw = remove_extreme(tweezers_rec, pairs_adjusted_t_nh) #we don't want to keep event when the other tool is in the glass

    """Select points"""
    mask_glass_nh = mask_extreme_xpos_event_removed_nh & mask_extreme_v_event_removed_nh & selected_xpos_nh
    mask_glass_tw = mask_extreme_xpos_event_removed_tw & mask_extreme_v_event_removed_tw & selected_xpos_tw
    selected_points_nh = needle_holder_rec[mask_glass_nh & mask_extreme_v_glass_nh] #v is the most discriminative in this case compared to xpos
    selected_points_tw = tweezers_rec[mask_glass_tw & mask_extreme_v_glass_tw] #v is the most discriminative in this case compared to xpos

    #so far we excluded events where the tool is reaching the glass, we then want a more precise selection of points
    """2nd selection based on z position"""
    std_nh =  (selected_points_nh[['X.1', 'Y.1', 'Z.1']]).std()
    std_tw =  (selected_points_tw[['X.1', 'Y.1', 'Z.1']]).std()
    if (subject == 19): #subject 19 is treated separately, two different ref level of z position during the task 
        selected_zpos_nh, selected_zpos_tw = remove_extreme_using_z_pos2_sub19(needle_holder_rec, tweezers_rec)
    elif ((subject == 23) or (subject==27)):
        selected_zpos_nh, selected_zpos_tw = remove_extreme_using_z_pos_exception(needle_holder_rec, tweezers_rec, std_nh, std_tw, medians_nh, medians_tw)
    else: 
        selected_zpos_nh, selected_zpos_tw = remove_extreme_using_z_pos2(needle_holder_rec, tweezers_rec, std_nh, std_tw, medians_nh, medians_tw)
    
    time_event_zpos_nh, time_event_zpos_tw = compute_event_time_pos(full_segments_time[i], needle_holder_rec, tweezers_rec, selected_zpos_nh,
                                                                selected_zpos_tw)

    pairs_t_zpos_nh = compute_pairs_time(time_event_zpos_nh[0], time_event_zpos_nh[1])
    pairs_t_zpos_tw = compute_pairs_time(time_event_zpos_tw[0], time_event_zpos_tw[1])

    pairs_adjusted_t_zpos_nh = adjust_time_pair(selected_points_nh, pairs_t_zpos_nh, medians_nh['Z.1'], time_allowed = 6)
    pairs_adjusted_t_zpos_tw = adjust_time_pair(selected_points_tw, pairs_t_zpos_tw, medians_tw['Z.1'], time_allowed=6)

    mask_extreme_zpos_event_removed_nh = remove_extreme(selected_points_nh, pairs_adjusted_t_zpos_nh)
    mask_extreme_zpos_event_removed_tw = remove_extreme(selected_points_tw, pairs_adjusted_t_zpos_tw)

    
    selected_points_nh2 = selected_points_nh[mask_extreme_zpos_event_removed_nh]
    selected_points_tw2 = selected_points_tw[mask_extreme_zpos_event_removed_tw]

    """2nd selection based on y position"""
    if ((subject == 23) or (subject==27)):
        selected_ypos_nh, selected_ypos_tw = remove_extreme_using_y_pos_exception(needle_holder_rec, tweezers_rec, std_nh, std_tw, medians_nh, medians_tw)
    else:
        selected_ypos_nh, selected_ypos_tw = remove_extreme_using_y_pos(needle_holder_rec, tweezers_rec, std_nh, std_tw, medians_nh, medians_tw)
    
    time_event_ypos_nh, time_event_ypos_tw = compute_event_time_pos(full_segments_time[i], needle_holder_rec, tweezers_rec, selected_ypos_nh,
                                                                selected_ypos_tw)

    pairs_t_ypos_nh = compute_pairs_time(time_event_ypos_nh[0], time_event_ypos_nh[1])
    pairs_t_ypos_tw = compute_pairs_time(time_event_ypos_tw[0], time_event_ypos_tw[1])

    pairs_adjusted_t_ypos_nh = adjust_time_pair(selected_points_nh2, pairs_t_ypos_nh, medians_nh['Y.1'], time_allowed = 6)
    pairs_adjusted_t_ypos_tw = adjust_time_pair(selected_points_tw2, pairs_t_ypos_tw, medians_tw['Y.1'], time_allowed=6)

    mask_extreme_ypos_event_removed_nh = remove_extreme(selected_points_nh2, pairs_adjusted_t_ypos_nh)
    mask_extreme_ypos_event_removed_tw = remove_extreme(selected_points_tw2, pairs_adjusted_t_ypos_tw)

    
    selected_points_nh3 = selected_points_nh2[mask_extreme_ypos_event_removed_nh]
    selected_points_tw3 = selected_points_tw2[mask_extreme_ypos_event_removed_tw]

    """2nd selection based on x position"""
    #x position ref line of subject 1 is moving during the task, we need to treat it separately
    """if (subject!=1):
        selected_xpos2_nh, selected_xpos2_tw = remove_extreme_using_x_pos_on_selected_data_exception(needle_holder_rec, tweezers_rec, std_nh, std_tw, medians_nh, medians_tw)
    else:
        selected_xpos2_nh, selected_xpos2_tw  = remove_extreme_using_x_pos_on_selected_data_sub1(needle_holder_rec, tweezers_rec, selected_points_nh, selected_points_tw)
    """
    if (subject==1):
        selected_xpos2_nh, selected_xpos2_tw  = remove_extreme_using_x_pos_on_selected_data_sub1(needle_holder_rec, tweezers_rec, selected_points_nh, selected_points_tw)
    elif ((subject == 23) or (subject==27)):
        selected_xpos2_nh, selected_xpos2_tw = remove_extreme_using_x_pos_on_selected_data_exception(needle_holder_rec, tweezers_rec, std_nh, std_tw, medians_nh, medians_tw)
    else:
        selected_xpos2_nh, selected_xpos2_tw = remove_extreme_using_x_pos_on_selected_data(needle_holder_rec, tweezers_rec, std_nh, std_tw, medians_nh, medians_tw)
    
    time_event_xpos2_nh, time_event_xpos2_tw = compute_event_time_pos(full_segments_time[i], needle_holder_rec, tweezers_rec, selected_xpos2_nh,
                                                                selected_xpos2_tw)

    pairs_t_xpos2_nh = compute_pairs_time(time_event_xpos2_nh[0], time_event_xpos2_nh[1])
    pairs_t_xpos2_tw = compute_pairs_time(time_event_xpos2_tw[0], time_event_xpos2_tw[1])

    pairs_adjusted_t_xpos2_nh = adjust_time_pair(selected_points_nh3, pairs_t_xpos2_nh, medians_nh['X.1'], time_allowed = 6)
    pairs_adjusted_t_xpos2_tw = adjust_time_pair(selected_points_tw3, pairs_t_xpos2_tw, medians_tw['X.1'], time_allowed=6)

    mask_extreme_xpos2_event_removed_nh = remove_extreme(selected_points_nh3, pairs_adjusted_t_xpos2_nh)
    mask_extreme_xpos2_event_removed_tw = remove_extreme(selected_points_tw3, pairs_adjusted_t_xpos2_tw)

    selected_points_nh4 = selected_points_nh3[mask_extreme_xpos2_event_removed_nh]
    selected_points_tw4 = selected_points_tw3[mask_extreme_xpos2_event_removed_tw]

    """Manual correction of incorrectly segmented points """
    if (subject==1):
        mask_tw = (selected_points_tw4['Time (Seconds)']>1275) & (selected_points_tw4['Time (Seconds)']<1287)
        selected_points_tw4 = selected_points_tw4[~mask_tw]
    if (subject==13):
        mask_tw = (selected_points_tw4['Time (Seconds)']>679) & (selected_points_tw4['Time (Seconds)']<711.5)
        selected_points_tw4 = selected_points_tw4[~mask_tw]
    """end"""
    return selected_points_nh4, selected_points_tw4

In [97]:
subject=1
i=0

selected_nh_d,selected_tw_d = run_step1_per_subject2(subject,i, new_f=5)
#selected_nh,selected_tw = run_step1_per_subject(subject,i)

#apply filter to coordinates
"""selected_nh_d['X.1'] = butter_lowpass_filter(selected_nh_d['X.1'], cutoff=12) #changer la fréquence initiale
selected_nh_d['Y.1'] = butter_lowpass_filter(selected_nh_d['Y.1'], cutoff=12)
selected_nh_d['Z.1'] = butter_lowpass_filter(selected_nh_d['Z.1'], cutoff=12)

selected_tw_d['X.1'] = butter_lowpass_filter(selected_tw_d['X.1'], cutoff=12)
selected_tw_d['Y.1'] = butter_lowpass_filter(selected_tw_d['Y.1'], cutoff=12)
selected_tw_d['Z.1'] = butter_lowpass_filter(selected_tw_d['Z.1'], cutoff=12)"""

#cut the time points in stitches
dict_segment_time = full_segments_time[i]
list_np_segmented_nh_rec = pd_2_numpy_and_segment(selected_nh_d, dict_segment_time)
#test = pd_2_numpy_and_segment(selected_nh, dict_segment_time)
list_np_segmented_tw_rec = pd_2_numpy_and_segment(selected_tw_d, dict_segment_time)


In [98]:
for s in range(8):
    t_tw = list_np_segmented_tw_rec[s][:,0]
    path_length_tw = get_path_length_g(t_tw, list_np_segmented_tw_rec[s][:,1:4])
    print(path_length_tw)

3.6449977880804676
1.7303243762786447
2.4823588932976506
1.72815224528555
1.8133463577487907
0.9675961298887499
1.258706876984081
2.763499147579105


In [99]:
df_metrics = get_features_simplified(subject, i, list_np_segmented_tw_rec, list_np_segmented_nh_rec, save='False')

Save


In [94]:
df_metrics

,Tool,Stitch,Effective_task_duration,Idle_time,Path_length,Jerk,Mean_v,Std_v,Economy_of_volume,Mean_curvature,Std_curvature,Angular_displacement,Rate_of_orientation_change,Bimanual_dexterity,Bimanual_efficacy
0,TW,0,221.0,6298.0,3.644998,-8.808816,0.015598,0.015931,0.016366,-4.142510e+02,6.666598e+04,2318.312033,NaN,None,NaN
1,NH,0,125.6,6393.4,1.105117,-7.042471,0.008540,0.010029,0.032096,1.536399e+02,4.223318e+04,1304.016673,NaN,None,NaN
2,TW,1,129.6,3932.8,1.730324,-7.291495,0.012925,0.014722,0.027794,2.097226e+03,7.831455e+04,1371.226526,NaN,None,NaN
3,NH,1,68.8,3993.4,0.893327,-5.413692,0.011742,0.011881,0.033530,-3.708655e+02,3.042657e+04,712.665659,NaN,None,NaN
4,TW,2,215.2,5395.2,2.482359,-8.711269,0.011146,0.012906,0.020769,-5.611928e+03,1.516916e+05,2273.909911,NaN,None,NaN
5,NH,2,209.4,5401.0,2.359196,-8.734658,0.010947,0.012593,0.014734,-2.765872e+02,4.042681e+04,2184.748064,NaN,None,NaN
6,TW,3,118.6,7167.8,1.728152,-7.022113,0.013693,0.013751,0.026127,-9.339637e+04,3.845596e+06,1249.095326,NaN,None,NaN
7,NH,3,111.4,7175.0,1.246557,-6.794240,0.010747,0.012621,0.020571,1.363894e+03,4.271875e+04,1152.268128,NaN,None,NaN
8,TW,4,143.4,4632.6,1.813346,-7.628585,0.012286,0.014144,0.018829,4.532355e+02,1.181297e+05,1519.844175,NaN,None,NaN
9,NH,4,140.6,4635.4,1.360903,-7.363543,0.009247,0.009573,0.023537,7.695462e+03,2.414456e+05,1468.839635,NaN,None,NaN


In [26]:
subjects=[1,19, 23,7, 24 , 26, 10, 13, 16, 17, 20, 27]
i=0
for subject in subjects:
    selected_nh_d,selected_tw_d = run_step1_per_subject2(subject,i, new_f=5)

    #cut the time points in stitches
    dict_segment_time = full_segments_time[i]
    list_np_segmented_nh_rec = pd_2_numpy_and_segment(selected_nh_d, dict_segment_time)
    #test = pd_2_numpy_and_segment(selected_nh, dict_segment_time)
    list_np_segmented_tw_rec = pd_2_numpy_and_segment(selected_tw_d, dict_segment_time)

    df_metrics = get_features_simplified(subject, i, list_np_segmented_tw_rec, list_np_segmented_nh_rec, save='False')
    print(subject)
    print(df_metrics)
    i=i+1

Save
1
   Tool  Stitch  Effective_task_duration  Idle_time  Path_length      Jerk  \
0    TW       0                    221.0     7562.4     3.644998 -8.808816   
1    NH       0                    125.6     7657.6     1.105117 -7.042471   
2    TW       1                    129.6     5133.8     1.730324 -7.291495   
3    NH       1                     68.8     5194.4     0.893327 -5.413692   
4    TW       2                    215.2     8614.6     2.482359 -8.711269   
5    NH       2                    209.4     8620.2     2.359196 -8.734658   
6    TW       3                    118.6     5852.8     1.728152 -7.022113   
7    NH       3                    111.4     5859.8     1.246557 -6.794240   
8    TW       4                    143.4     5384.0     1.813346 -7.628585   
9    NH       4                    140.6     5386.6     1.360903 -7.363543   
10   TW       5                     56.8     3176.2     0.967596 -4.638839   
11   NH       5                     49.8     3183.0     0